## 5. Video Processing Pipeline   

* Image undistort
* Color transform & level thresholding
* Gradient detector for edges
* Perspective transform to dewarp camera image
* Lane detect using historical tracking data
* Calculate curvature metrics
* Warp back to original image geometry

In [3]:
#Importing packages required for notebook
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from IPython.display import Markdown
import cv2
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline
print("Packages imported ...")

# Load Camera Calibration Data
cameraData = pickle.load( open('./Resources/pickled_data/camera_calibration.p', 'rb' ) )
mtx, dist = map(cameraData.get, ('mtx', 'dist'))
print("Loaded camera calibration data from file ...")

# Load camera warp data
transMatrix = pickle.load( open('./Resources/pickled_data/perspective_transform.p', 'rb' ) )
M, Minv = map(transMatrix.get, ('M', 'Minv'))
print("Loaded camera warp data from file ...")

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# global array to hold lane indicators across video frames
prev_left_lane_inds = []
prev_right_lane_inds = []

# Undistort image frame usign camera distorition matrix
def image_undistort (image):
    return cv2.undistort(image, mtx, dist, None, mtx) 
    
# Transform color space to HSL+RGB --> RHS component space
def color_transform (image):
    hls_img =cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #return np.dstack((rgb_img[:,:,0], hls_img[:,:,0], hls_img[:,:,2]))
    #return np.dstack(( hls_img[:,:,2], hls_img[:,:,2], hls_img[:,:,2]))
    return hls_img

# Select white+yellow colors from RGB & HSL spaces, threshold to a binary image
def color_select (image):
    # Color space transforms
    img_hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    # RGB, HLS & Lab color thresholds for yellow & white 
    # combined yellow & white thresholds
    lowrgb_white = np.array([100,100,200], dtype = "uint8")
    highrgb_white = np.array([255,255,255], dtype = "uint8")
    lowrgb_yellow = np.array([255,180,0], dtype = "uint8")
    highrgb_yellow = np.array([255,255,170], dtype = "uint8")
    lowhls_yellow = np.array([20,120,130], dtype = "uint8")
    highhls_yellow = np.array([45,200,255], dtype = "uint8")
    lowlab_yellow = np.array([0,0,154], dtype = "uint8")
    highlab_yellow = np.array([0,0,255], dtype = "uint8")
    
    # RGB white
    mask = cv2.inRange(img_rgb, lowrgb_white, highrgb_white)
    rgb_ww = cv2.bitwise_and(img_rgb, img_rgb, mask = mask).astype(np.uint8)
    rgb_ww = cv2.cvtColor(rgb_ww, cv2.COLOR_RGB2GRAY)
    rgb_w = np.zeros_like(rgb_ww)
    rgb_w[(rgb_ww >= 80) & (rgb_ww <= 255)] = 1
    
    # RGB yellow
    mask = cv2.inRange(img_rgb, lowrgb_yellow, highrgb_yellow)
    rgb_yy = cv2.bitwise_and(img_rgb, img_rgb, mask = mask).astype(np.uint8)
    rgb_yy = cv2.cvtColor(rgb_yy, cv2.COLOR_RGB2GRAY)
    rgb_y = np.zeros_like(rgb_yy)
    rgb_y[(rgb_yy >= 20) & (rgb_yy <= 255)] = 1

    # HLS yellow
    mask = cv2.inRange(img_hls, lowhls_yellow, highhls_yellow)
    hls_yy = cv2.bitwise_and(img_hls, img_hls, mask = mask).astype(np.uint8)
    hls_yy = cv2.cvtColor(hls_yy, cv2.COLOR_HLS2RGB)
    hls_yy = cv2.cvtColor(hls_yy, cv2.COLOR_RGB2GRAY)
    hls_y = np.zeros_like(hls_yy)
    hls_y[(hls_yy >= 50) & (hls_yy <= 255)] = 1

    # Lab yellow
    mask = cv2.inRange(img_lab, lowlab_yellow, highlab_yellow)
    lab_yy = cv2.bitwise_and(img_lab, img_lab, mask = mask).astype(np.uint8)
    lab_yy = cv2.cvtColor(lab_yy, cv2.COLOR_HLS2RGB)
    lab_yy = cv2.cvtColor(lab_yy, cv2.COLOR_RGB2GRAY)
    lab_y = np.zeros_like(lab_yy)
    lab_y[(lab_yy >= 50) & (lab_yy <= 255)] = 1

    # combined yellow+white thresholds
    bin_whiteyellow = np.zeros_like(hls_y)
    bin_whiteyellow[(hls_y == 1) | (rgb_y == 1) | (rgb_w == 1) | (lab_y == 1)] = 255

    return np.dstack(( bin_whiteyellow, bin_whiteyellow, bin_whiteyellow))


# Level threshold RHS componenets per tuned threhold values
def color_threshold (image):
    r_thresh_min = 90
    r_thresh_max = 145
    h_thresh_min = 20
    h_thresh_max = 45
    s_thresh_min = 80
    s_thresh_max = 255    
    r_channel = image[:,:,0]
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh_min) & (r_channel <= r_thresh_max)] = 1
    h_channel = image[:,:,1]
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh_min) & (h_channel <= h_thresh_max)] = 1
    s_channel = image[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
 
    return (np.dstack(( r_binary, h_binary, s_binary))*255)
    
# Process RHS component images 
#  apply Sobel/Laplacian gradient thresholding (absolute, magnitude & angle)
def gradient_detect (s_channel, filter_type="sobel", kernel_size=7, abs_thresh=(0, 255), mag_thresh=(0, 255), angle_thresh=(0, np.pi/2)):
    
    # absolute thresholding
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if (filter_type == "sobel"):
        sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0, ksize=kernel_size)
        sobely = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1, ksize=kernel_size)
    else: #"laplacian"
        sobelx = cv2.Laplacian(s_channel, cv2.CV_64F, ksize=kernel_size)
        sobely = cv2.Laplacian(s_channel, cv2.CV_64F, ksize=kernel_size)
    # Rescale back to 8 bit integer
    scaled_sobelx = np.uint8(255*np.absolute(sobelx)/np.max(np.absolute(sobelx)))
    scaled_sobely = np.uint8(255*np.absolute(sobely)/np.max(np.absolute(sobely)))
    # Create a copy and apply the threshold
    abs_binary_x = np.zeros_like(scaled_sobelx)
    abs_binary_y = np.zeros_like(scaled_sobely)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    abs_binary_x[(scaled_sobelx >= abs_thresh[0]) & (scaled_sobelx <= abs_thresh[1])] = 1
    abs_binary_y[(scaled_sobely >= abs_thresh[0]) & (scaled_sobely <= abs_thresh[1])] = 1

    # magnitude thresholding
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # angle thresholding
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= angle_thresh[0]) & (absgraddir <= angle_thresh[1])] = 1

    # generate combined thresholded S-channel image
    combined = np.zeros_like(dir_binary)
    combined[((abs_binary_x == 1) & (abs_binary_y == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 255

    return combined.astype(np.uint8)

def gradient_threshold (image):
    #resultR = gradient_detect(image[:,:,0], filter_type="laplacian", abs_thresh=(20, 160), mag_thresh=(20, 160), angle_thresh=(0.5, np.pi/2))
    #resultH = gradient_detect(image[:,:,1], filter_type="laplacian", abs_thresh=(20, 160), mag_thresh=(20, 160), angle_thresh=(0.5, np.pi/2))
    resultS = gradient_detect(image[:,:,2], filter_type="sobel", abs_thresh=(30, 255), mag_thresh=(20, 255), angle_thresh=(0.5, np.pi/2))
    result = np.zeros_like(resultS)
    #result[((resultR == 255) & (resultH == 255)) | (resultS == 255)] = 255
    result = resultS
    
    return np.dstack((result, result, result))
    #return result.astype(np.uint8)

def warp_image (image):
    return cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))

# Lane line detect using sliding-window histogram tracking
def find_lane_pixels (binary_warped, prev_left_lane_inds, prev_right_left_lane_inds):
    # Take a histogram of the bottom third of the image 
    #  (guard against curved lanes by looking at immediate road lines next to car front)
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/3):,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 110
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        # Re-use lane indicators from last available frame
        left_lane_inds = prev_left_lane_inds
        right_lane_inds = prev_right_lane_inds
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img, left_lane_inds, right_lane_inds

def fit_polynomial (binary_warped, prev_left_lane_inds, prev_right_left_lane_inds):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img, prev_left_lane_ids, prev_right_lane_inds = find_lane_pixels(binary_warped, prev_left_lane_inds, prev_right_left_lane_inds)

    idx=0
    #print(leftx[idx], lefty[idx], rightx[idx], righty[idx])
    
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        left_fitx_m= left_fit_m[0]*ploty**2 + left_fit_m[1]*ploty + left_fit_m[2]
        right_fitx_m = right_fit_m[0]*ploty**2 + right_fit_m[1]*ploty + right_fit_m[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        left_fitx_m = 1*ploty**2 + 1*ploty
        right_fitx_m = 1*ploty**2 + 1*ploty
    center_fitx = np.mean([left_fit[0],right_fit[0]])*ploty**2 + np.mean([left_fit[1],right_fit[1]])*ploty + np.mean([left_fit[2],right_fit[2]])

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0] # red for left lane
    out_img[righty, rightx] = [0, 0, 255] # blue for right lane
    ## draw lanelines & center track
    #out_img[ploty.astype(np.int64), left_fitx.astype(np.int64)] = [255, 255, 0] # yellow for left polynomial fit
    #out_img[ploty.astype(np.int64), right_fitx.astype(np.int64)] = [255, 255, 0] # yellow for right polynomial fit
    #out_img[ploty.astype(np.int64), center_fitx.astype(np.int64)] = [255, 255, 255] # yellow for road center 

    return out_img, ploty, left_fitx, right_fitx, center_fitx, left_fit_m, right_fit_m

def measure_curvature_real(ploty, left_fit_cr, right_fit_cr):
    '''
    Calculates the curvature of polynomial functions in kilometers.
    '''
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    return left_curverad/1000., right_curverad/1000.

def draw_lane_lines (img, left_fitx, right_fitx):
    yMax = img.shape[0]
    ploty = np.linspace(0, yMax - 1, yMax)
    color_warp = np.zeros_like(img).astype(np.uint8)
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    return cv2.addWeighted(img, 1, newwarp, 0.3, 0)

def annotate_lanes (img, left_fit_m, right_fit_m, leftCurvature, rightCurvature, fontScale=.75):
    # Calculate vehicle center
    xMax = img.shape[1]*xm_per_pix
    yMax = img.shape[0]*ym_per_pix
    vehicleCenter = xMax / 2
    
    lineLeft = left_fit_m[0]*yMax**2 + left_fit_m[1]*yMax + left_fit_m[2]
    lineRight = right_fit_m[0]*yMax**2 + right_fit_m[1]*yMax + right_fit_m[2]
    lineMiddle = lineLeft + (lineRight - lineLeft)/2
    diffFromVehicle = lineMiddle - vehicleCenter
    if diffFromVehicle > 0:
        message = '{:.2f} m right'.format(diffFromVehicle)
    else:
        message = '{:.2f} m left'.format(-diffFromVehicle)
    
    # Draw info
    font = cv2.FONT_HERSHEY_COMPLEX
    fontColor = (124, 252, 0) #(255, 255, 255)
    cv2.putText(img, 'Left curvature: {:.0f} m'.format(leftCurvature), (50, 50), font, fontScale, fontColor, 2)
    cv2.putText(img, 'Right curvature: {:.0f} m'.format(rightCurvature), (50, 75), font, fontScale, fontColor, 2)
    cv2.putText(img, 'Vehicle is {} of center'.format(message), (50, 100), font, fontScale, fontColor, 2)
     
    return img    
    
def draw_lanes_on_image (img, left_fitx, right_fitx, left_fit_m, right_fit_m, leftCurvature, rightCurvature):
    
    yMax = img.shape[0]
    ploty = np.linspace(0, yMax - 1, yMax)
    color_warp = np.zeros_like(img).astype(np.uint8)
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 

    return cv2.addWeighted(img, 1, newwarp, 0.3, 0)


def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width=50, window_height=50, margin=100):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
	    # convolve the window into the vertical slice of the image
	    image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
	    conv_signal = np.convolve(window, image_layer)
	    # Find the best left centroid by using past left center as a reference
	    # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
	    offset = window_width/2
	    l_min_index = int(max(l_center+offset-margin,0))
	    l_max_index = int(min(l_center+offset+margin,image.shape[1]))
	    l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
	    # Find the best right centroid by using past right center as a reference
	    r_min_index = int(max(r_center+offset-margin,0))
	    r_max_index = int(min(r_center+offset+margin,image.shape[1]))
	    r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
	    # Add what we found for that layer
	    window_centroids.append((l_center,r_center))

    return window_centroids

def roi_mask (img, maskfit_factor_horiz=3.1, maskfit_factor_vert=1.8):
    # define region of interest, 4-sided polygon
    ysize = img.shape[0]
    xsize = img.shape[1]
    offset_height = ysize/4 # 1/4th bottom is always kept unmasked
    left_end = np.array([0, ysize])
    right_end = np.array([xsize, ysize])
    left_bottom = np.array([0, ysize-offset_height])
    right_bottom = np.array([xsize, ysize-offset_height])
    # check ROI top vertices don't cross
    if (xsize/maskfit_factor_horiz < xsize/2):
        left_top = np.array([xsize/maskfit_factor_horiz, ysize/maskfit_factor_vert])
        right_top = np.array([xsize-xsize/maskfit_factor_horiz, ysize/maskfit_factor_vert])
    else:
        left_top = np.array([xsize/2, ysize/maskfit_factor_vert])
        right_top = np.array([xsize/2, ysize/maskfit_factor_vert])
    roi_vertices = np.array([[left_end,left_bottom,left_top,right_top,right_bottom,right_end]], dtype=np.int32)
    
    # apply mask
    mask = np.zeros_like(img)   
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, roi_vertices, ignore_mask_color)
    #returning the image only where mask pixels are nonzero
    img = cv2.bitwise_and(img, mask)

    return img

def VideoLaneTrackingPipeline(image):
    #
    # Camera distortion model processing
    #
    img = image_undistort(image)
    
    #
    # Color processing
    #
    if (0):
        img = color_transform(img)
        img = color_threshold(img)     
    else:
        img = color_select (img)

    #
    # RoI Masking & Gradient processing
    #
    img = roi_mask(img)
    img = gradient_threshold(img)
    
    #
    # Lane marker detection & tracking    
    #
    img = warp_image(img)
    img, ploty, l_fitx, r_fitx, lr_fitx, l_fit_m, r_fit_m = fit_polynomial(img[:,:,2], prev_left_lane_inds, prev_left_lane_inds)
    left_curverad, right_curverad = measure_curvature_real(ploty, l_fit_m, r_fit_m)
    #print('Left : {:.2f} km, Right : {:.2f} km'.format(left_curverad, right_curverad))
    
    #
    # Video frame annotation
    #
    img = draw_lane_lines(image, l_fitx, r_fitx)
    ##img = draw_lanes_on_image (image, l_fitx, r_fitx, l_fit_m, r_fit_m, left_curverad, right_curverad)
    img = annotate_lanes(img, l_fit_m, r_fit_m, left_curverad, right_curverad)

    return img

"""
#
# Process images in test directory
#
out_images = []
files = os.listdir("./test_images/")
for file in files:
    iimg = cv2.imread("./test_images/"+file)
    oimg = VideoLaneTrackingPipeline(iimg)
    out_images.append(oimg)
    cv2.imwrite('./output_images/'+file, oimg) 

imgLength = len(out_images)
rows = 2
cols = int(len(out_images)/rows)
fig, axes = plt.subplots(rows, cols, figsize=(15,4))
plt.suptitle('Test Images With Lane Markers')
plt.subplots_adjust(wspace=0.1, hspace=0.1)
#plt.tight_layout()
indexes = range(cols * rows)
for ax, index in zip(axes.flat, indexes):    
    ax.imshow(cv2.cvtColor(out_images[index], cv2.COLOR_BGR2RGB))
    #ax.imshow(out_images[index], cmap='gray')
    ax.axis('off')
"""
    

#
# Process videos in test directory
#
files = os.listdir("./Resources/test_videos/")
for file in files:
    video_output = "./Resources/test_videos_output/"+file    
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    clip1 = VideoFileClip("./Resources/test_videos/"+file).subclip(5,10)
    #clip1.reader.close()
    #clip1.audio.reader.close_proc()
    #clip1.close()
    video_clip = clip1.fl_image(VideoLaneTrackingPipeline) #NOTE: this function expects color images!!
    #video_clip = clip1
    %time video_clip.write_videofile(video_output, audio=False)
    del clip1
    

Packages imported ...
Loaded camera calibration data from file ...
Loaded camera warp data from file ...
[MoviePy] >>>> Building video ./Resources/test_videos_output/project_video.mp4
[MoviePy] Writing video ./Resources/test_videos_output/project_video.mp4


 99%|███████████████████████████████████████████████████████████████████████████████▎| 125/126 [00:55<00:00,  2.26it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./Resources/test_videos_output/project_video.mp4 

Wall time: 57.3 s
